In [ ]:
# python notebook for Make Your Own Neural Network
# code for a 3-layer neural network, and code for learning the MNIST dataset
# (c) Tariq Rashid, 2016
# license is GPLv2

In [ ]:
# import numpy as np
import numpy as np
# scipy.special for the sigmoid function expit()
import scipy.special
# library for plotting arrays
import matplotlib.pyplot
import math
# ensure the plots are inside this notebook, not an external window
%matplotlib inline

In [ ]:
# neural network class definition
class neuralNetwork:
    
    
    # initialise the neural network
    def __init__(self, inputnodes, hiddennodes, hidden2nodes, outputnodes, learningrate):
        # set number of nodes in each input, hidden, output layer
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.h2nodes = hidden2nodes
        self.onodes = outputnodes
        
        # link weight matrices, wih and who
        # weights inside the arrays are w_i_j, where link is from node i to node j in the next layer
        # w11 w21
        # w12 w22 etc 
        # self.wih = np.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes, self.inodes))
        # self.who = np.random.normal(0.0, pow(self.hnodes, -0.5), (self.onodes, self.hnodes))

        self.wih = (np.random.rand(self.hnodes, self.inodes) * 2) -1 
        self.whh2 = (np.random.rand(self.h2nodes, self.hnodes) * 2) -1 
        self.wh2o = (np.random.rand(self.onodes, self.h2nodes) * 2) -1


        # gleichverteilt zwischen -1/sqrt(hnodes) und 1/sqrt(hnodes)
        # self.wih = (((np.random.rand(self.hnodes, self.inodes)*2) / math.sqrt(self.hnodes))-(1/math.sqrt(self.hnodes)))
        # self.who =  (((np.random.rand(self.onodes, self.hnodes)*2) / math.sqrt(self.hnodes))-(1/math.sqrt(self.hnodes)))

        # learning rate
        self.lr = learningrate
        
        # activation function is the sigmoid function
        self.activation_function = lambda x: scipy.special.expit(x)
        
        pass

    
    # train the neural network
    #@jit(nopython=True) # Set "nopython" mode for best performance, equivalent to @njit
    def train(self, inputs_list, targets_list):
        # convert inputs list to 2d array
        inputs = np.array(inputs_list, ndmin=2).T
        targets = np.array(targets_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = np.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        hidden2_inputs = np.dot(self.whh2, hidden_outputs)
        hidden2_outputs = self.activation_function(hidden2_inputs)

        # calculate signals into final output layer
        final_inputs = np.dot(self.wh2o, hidden2_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        # output layer error is the (target - actual)
        output_errors = targets - final_outputs
        # hidden layer error is the output_errors, split by weights, recombined at hidden nodes
        hidden2_errors = np.dot(self.wh2o.T, output_errors) 
        hidden_errors = np.dot(self.whh2.T, hidden2_errors) 
        
        print(self.wih.shape)
        print(self.whh2.shape)
        print(self.wh2o.shape)

        # update the weights for the links between the hidden and output layers
        self.wh2o += self.lr * np.dot((output_errors * final_outputs * (1.0 - final_outputs)), np.transpose(hidden2_outputs))

        self.whh2 += self.lr * np.dot((hidden2_errors * final_outputs * (1.0 - final_outputs)), np.transpose(hidden_outputs))
        
        # update the weights for the links between the input and hidden layers
        self.wih += self.lr * np.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), np.transpose(inputs))
        
        
        pass

    
    # query the neural network
    def query(self, inputs_list):

        inputs = np.array(inputs_list, ndmin=2).T

        hidden_inputs = np.dot(self.wih, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)

        hidden2_inputs = np.dot(self.whh2, hidden_outputs)
        hidden2_outputs = self.activation_function(hidden2_inputs)

        final_inputs = np.dot(self.wh2o, hidden2_outputs)
        final_outputs = self.activation_function(final_inputs)

        return final_outputs 
        

In [ ]:
# number of input, hidden and output nodes
input_nodes = 784
hidden_nodes = 200
hidden2_nodes = 100
output_nodes = 10

# learning rate
learning_rate = 0.05

# create instance of neural network
n = neuralNetwork(input_nodes,hidden_nodes, hidden2_nodes, output_nodes, learning_rate)

In [ ]:
# load the mnist training data CSV file into a list
training_data_file = open("../mnist_train.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

In [60]:
import time
# from numba import jit

# epochs is the number of times the training data set is used for training
epochs = 20

#@numba.jit(forceobj=True)
#def sum(x):
#    total = 0
#    for i in range(x.shape[0]):
#        total += x[i]
#    return total

#print(sum([3, 5]))

#@jit
for e in range(epochs):
    print("Epoche " + str(e))
    start = time.time()

    # go through all records in the training data set
    for record in training_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # scale and shift the inputs
        inputs = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = np.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)
        pass
    end = time.time()
    print("duration: " + str(end - start))



Epoche 0
(200, 784)
(100, 200)
(10, 100)


ValueError: operands could not be broadcast together with shapes (100,1) (10,1) 

In [ ]:
# load the mnist test data CSV file into a list
test_data_file = open("../mnist_test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

In [ ]:
# test the neural network

# scorecard for how well the network performs, initially empty
scorecard = []

# go through all the records in the test data set
for record in test_data_list:
    # split the record by the ',' commas
    all_values = record.split(',')
    # correct answer is first value
    correct_label = int(all_values[0])
    # scale and shift the inputs
    inputs = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    # query the network
    outputs = n.query(inputs)
    # the index of the highest value corresponds to the label
    label = np.argmax(outputs)
    # append correct or incorrect to list
    if (label == correct_label):
        # network's answer matches correct answer, add 1 to scorecard
        scorecard.append(1)
    else:
        # network's answer doesn't match correct answer, add 0 to scorecard
        scorecard.append(0)
        pass
    
    pass

In [ ]:
# calculate the performance score, the fraction of correct answers
scorecard_array = np.asarray(scorecard)
print ("performance = ", scorecard_array.sum() / scorecard_array.size)

performance =  0.923
